In [1]:

import pandas as pd
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn import metrics

import pickle
import math
import re
import enchant
import os
import glob
import numpy as np
np.random.seed(512)

In [2]:
xtrain = pd.read_csv('../MLFeatureTypeInference/Benchmark-Labeled-Data/data_train.csv')
xtest = pd.read_csv('../MLFeatureTypeInference/Benchmark-Labeled-Data/data_test.csv')


xtrain = xtrain.sample(frac=1,random_state=100).reset_index(drop=True)
print(len(xtrain))

y_train = xtrain.loc[:,['y_act']]
y_test = xtest.loc[:,['y_act']]

7936


In [3]:
dict_label = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_train['y_act'] = [dict_label[i] for i in y_train['y_act']]
y_test['y_act'] = [dict_label[i] for i in y_test['y_act']]
y_train


,y_act
0,0
1,0
2,0
3,1
4,0
...,...
7931,0
7932,1
7933,8
7934,7


In [4]:
useStats = 1
useAttributeName = 1
useSample1 = 0
useSample2 = 0
## Using descriptive stats and attribute name

In [5]:
def ProcessStats(data,y):

    data1 = data[['total_vals', 'num_nans', '%_nans', 'num_of_dist_val', '%_dist_val', 'mean', 'std_dev', 'min_val', 'max_val','has_delimiters', 'has_url', 'has_email', 'has_date', 'mean_word_count',
       'std_dev_word_count', 'mean_stopword_total', 'stdev_stopword_total',
       'mean_char_count', 'stdev_char_count', 'mean_whitespace_count',
       'stdev_whitespace_count', 'mean_delim_count', 'stdev_delim_count',
       'is_list', 'is_long_sentence']]
    data1 = data1.reset_index(drop=True)
    data1 = data1.fillna(0)

    y.y_act = y.y_act.astype(float)
    
    return data1


vectorizerName = CountVectorizer(ngram_range=(2, 2), analyzer='char')
vectorizerSample = CountVectorizer(ngram_range=(2, 2), analyzer='char')

def FeatureExtraction(data,data1,flag):

    arr = data['Attribute_name'].values
    arr = [str(x) for x in arr]
    
    arr1 = data['sample_1'].values
    arr1 = [str(x) for x in arr1]
    arr2 = data['sample_2'].values
    arr2 = [str(x) for x in arr2]
    arr3 = data['sample_3'].values
    arr3 = [str(x) for x in arr3]    
    print(len(arr1),len(arr2))
    if flag:
        X = vectorizerName.fit_transform(arr)
        X1 = vectorizerSample.fit_transform(arr1)
        X2 = vectorizerSample.transform(arr2)   
        
    else:
        X = vectorizerName.transform(arr)
        X1 = vectorizerSample.transform(arr1)
        X2 = vectorizerSample.transform(arr2)        
        
#     print(f"> Length of vectorized feature_names: {len(vectorizer.get_feature_names())}")

    attr_df = pd.DataFrame(X.toarray())
    sample1_df = pd.DataFrame(X1.toarray())
    sample2_df = pd.DataFrame(X2.toarray())
    print(len(data1),len(attr_df),len(sample1_df),len(sample2_df))

    if useSample1: data2 = sample1_df
    if useSample2: data2 = sample2_df    
    
    data2 = pd.concat([data1, attr_df], axis=1, sort=False)
    print(len(data2))
    return data2

In [14]:
xtrain1 = ProcessStats(xtrain,y_train)
xtest1 = ProcessStats(xtest,y_test)


X_train = FeatureExtraction(xtrain,xtrain1,1)
X_test = FeatureExtraction(xtest,xtest1,0)


X_train_new = X_train.reset_index(drop=True)
y_train_new = y_train.reset_index(drop=True)
X_train_new = X_train_new.values
y_train_new = y_train_new.values


k = 5
kf = KFold(n_splits=k,random_state = 100, shuffle=True)
avg_train_acc,avg_test_acc = 0,0

max_depth_grid = [5,10,25,50,100,250]
# criterion_grid = ['gini', 'entropy', 'log_loss']
criterion_grid = ['log_loss']

# n_estimators_grid = [25,50,75,100]
# max_depth_grid = [50,100]

avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
avgsc,avgsc_train,avgsc_hld = 0,0,0

i=0
for train_index, test_index in kf.split(X_train_new):
#     if i==1: break
    i=i+1
    X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
    y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
    y_test_cur = [str(val) for val in y_test_cur]
    X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)

    bestPerformingModel = DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=100)
    bestscore = 0
    print('='*10)
    for md in max_depth_grid:
        for cr in criterion_grid:
            clf = DecisionTreeClassifier(max_depth=md, criterion=cr, random_state=100)
            clf.fit(X_train_train, y_train_train.ravel())
            sc = clf.score(X_val, y_val)
            print(f"[max_depth: {md}, accuracy: {sc}]")
            if bestscore < sc:
                bestmd = md
                bestcr = cr
                bestscore = sc
                bestPerformingModel = clf

    bscr_train = bestPerformingModel.score(X_train_cur, y_train_cur)
    bscr = bestPerformingModel.score(X_test_cur, y_test_cur)
    bscr_hld = bestPerformingModel.score(X_test.to_numpy(), y_test)

    avgsc_train_lst.append(bscr_train)
    avgsc_lst.append(bscr)
    avgsc_hld_lst.append(bscr_hld)

    avgsc_train = avgsc_train + bscr_train    
    avgsc = avgsc + bscr
    avgsc_hld = avgsc_hld + bscr_hld

    # print()
    # print(f"> Best n_estimator: {bestne} || Best max_depth: {bestmd}")
    # print(f"> Best training score: {bscr_train}")
    # print(f"> Best test score: {bscr}")
    # print(f"> Best held score: {bscr_hld}")
print('='*10)

print(avgsc_train_lst)
print(avgsc_lst)
print(avgsc_hld_lst)

print(avgsc_train/k)
print(avgsc/k)
print(avgsc_hld/k)

y_pred = bestPerformingModel.predict(X_test.to_numpy())
bscr_hld = bestPerformingModel.score(X_test.to_numpy(), y_test)
print(bscr_hld)

7936 7936
7936 7936 7936 7936
7936
1985 1985
1985 1985 1985 1985
1985
[max_depth: 5, accuracy: 0.7939508506616257]
[max_depth: 10, accuracy: 0.8872085696282294]
[max_depth: 25, accuracy: 0.8979206049149339]
[max_depth: 50, accuracy: 0.8979206049149339]
[max_depth: 100, accuracy: 0.8979206049149339]
[max_depth: 250, accuracy: 0.8979206049149339]


/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


[max_depth: 5, accuracy: 0.7991183879093199]
[max_depth: 10, accuracy: 0.8904282115869018]
[max_depth: 25, accuracy: 0.9030226700251889]
[max_depth: 50, accuracy: 0.9030226700251889]
[max_depth: 100, accuracy: 0.9030226700251889]
[max_depth: 250, accuracy: 0.9030226700251889]


/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


[max_depth: 5, accuracy: 0.7959697732997482]
[max_depth: 10, accuracy: 0.8759445843828715]
[max_depth: 25, accuracy: 0.8872795969773299]
[max_depth: 50, accuracy: 0.8872795969773299]
[max_depth: 100, accuracy: 0.8872795969773299]
[max_depth: 250, accuracy: 0.8872795969773299]


...

[max_depth: 5, accuracy: 0.801007556675063]
[max_depth: 10, accuracy: 0.878463476070529]
[max_depth: 25, accuracy: 0.889168765743073]
[max_depth: 50, accuracy: 0.889168765743073]
[max_depth: 100, accuracy: 0.889168765743073]
[max_depth: 250, accuracy: 0.889168765743073]


/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


[max_depth: 5, accuracy: 0.77455919395466]
[max_depth: 10, accuracy: 0.8841309823677582]
[max_depth: 25, accuracy: 0.8835012594458438]
[max_depth: 50, accuracy: 0.8835012594458438]
[max_depth: 100, accuracy: 0.8835012594458438]
[max_depth: 250, accuracy: 0.8835012594458438]
[0.9743226212980466, 0.974956686092298, 0.9713340683572216, 0.9718065837139707, 0.9310127579146322]
[0.0, 0.0, 0.0, 0.0, 0.0]
[0.8931989924433249, 0.8987405541561713, 0.8942065491183879, 0.8806045340050378, 0.8831234256926952]
0.9646865434752337
0.0
0.8899748110831235


/home/adriana/Documents/ML-Data-Prep-Zoo/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.8831234256926952


In [15]:
print(bestPerformingModel.score(X_test.to_numpy(), y_test))

print(bestmd)
print(bestcr)
print(bestscore)

0.8831234256926952
10
log_loss
0.8841309823677582


In [16]:
from sklearn.metrics import f1_score, precision_score, recall_score


def PrintMetrics(y_true, y_pred):
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')

    matrix = confusion_matrix(y_true, y_pred)
    print(matrix)

    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 score: {f1}')


PrintMetrics(y_test, bestPerformingModel.predict(X_test.to_numpy()))

Accuracy: 0.8831234256926952
[[668  19   0   0   0   0   0   3  17]
 [ 29 409   0   1   1   2   2   6   7]
 [  2   0 138   0   0   0   0   0   1]
 [  0   6   0  71   0   0   6   6   3]
 [  0   2   0   1  28   0   1   0   0]
 [  1   3   1   0   0  89   2   0   3]
 [  0   4   0   3   0   2  46   1   1]
 [  8   4   0   1   0   0   0 197   5]
 [ 47  21   1   5   0   0   1   3 107]]
Precision: [0.88476821 0.87393162 0.98571429 0.86585366 0.96551724 0.95698925
 0.79310345 0.91203704 0.74305556]
Recall: [0.94483734 0.89496718 0.9787234  0.77173913 0.875      0.8989899
 0.80701754 0.91627907 0.57837838]
F1 score: [0.91381669 0.88432432 0.98220641 0.81609195 0.91803279 0.92708333
 0.8        0.91415313 0.65045593]


In [ ]:
gini

Accuracy: 0.8926952141057934
[[669  13   0   0   0   0   0   5  20]
 [ 16 418   0   5   1   1   0   2  14]
 [  0   0 138   0   0   0   0   1   2]
 [  0   2   0  81   1   0   2   3   3]
 [  0   0   0   0  31   0   1   0   0]
 [  1   5   2   0   0  87   1   0   3]
 [  0   1   1   3   0   1  44   0   7]
 [  6  12   4   1   0   0   0 188   4]
 [ 41  21   1   4   0   0   0   2 116]]
Precision: [0.91268759 0.88559322 0.94520548 0.86170213 0.93939394 0.97752809
 0.91666667 0.93532338 0.68639053]
Recall: [0.94625177 0.91466083 0.9787234  0.88043478 0.96875    0.87878788
 0.77192982 0.8744186  0.62702703]
F1 score: [0.92916667 0.89989236 0.96167247 0.87096774 0.95384615 0.92553191
 0.83809524 0.90384615 0.65536723]

In [ ]:
entropy

Accuracy: 0.8831234256926952
[[668  19   0   0   0   0   0   3  17]
 [ 29 409   0   1   1   2   2   6   7]
 [  2   0 138   0   0   0   0   0   1]
 [  0   6   0  71   0   0   6   6   3]
 [  0   2   0   1  28   0   1   0   0]
 [  1   3   1   0   0  89   2   0   3]
 [  0   4   0   3   0   2  46   1   1]
 [  8   4   0   1   0   0   0 197   5]
 [ 47  21   1   5   0   0   1   3 107]]
Precision: [0.88476821 0.87393162 0.98571429 0.86585366 0.96551724 0.95698925
 0.79310345 0.91203704 0.74305556]
Recall: [0.94483734 0.89496718 0.9787234  0.77173913 0.875      0.8989899
 0.80701754 0.91627907 0.57837838]
F1 score: [0.91381669 0.88432432 0.98220641 0.81609195 0.91803279 0.92708333
 0.8        0.91415313 0.65045593]